In [ ]:
import os
from langchain.agents import Tool
from langgraph.graph import StateGraph
from langchain_core.runnables import RunnableLambda
from typing import Any, Dict, List, Optional

# Libraries for OCR_branching_condition and OCR_Node
import None

# Libraries for Summary_Node
import None

# Libraries for Sending_Email_Node
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [ ]:
class NotifyState(dict):
    subscription: str               # from Crawl-Detect Engine
    receiver_emails: List[str]      # from Crawl-Detect Engine
    html_processed: str             # from Crawl-Detect Engine
    ocr_text: Optional[str]         # from OCR(optional) node
    json_summary: Dict[str, Any]    # from Summary node

In [ ]:
def OCR(state):
    #===================================================================
    # TO DO: 분기 조건에서 image 존재여부를 확인하여 추출 후 OCR 작업 수행
    image = None
    # image to ocr_text
    ocr_text = None
    #===================================================================

    return {'ocr_text':ocr_text}

def Summary(state):
    html = state.get('html_processed')
    ocr_text = state.get('ocr_text') or "" # ocr_text가 없다면 빈 문자열을 처리 예정
    #=================================================================================
    # TO DO: html_processed, ocr_text(optional)을 prompt 내용으로 포함하여 llm summary
    prompt = """\
    당신은 메일 전송을 위한 핵심요약 추출 전문가입니다.
    내용:
    {html_processed}
    {ocr_text}
    위 내용을 통합정리 및 요약해서 아래 json 형식에 맞추어 결과를 반환하세요.
    """
    llm = "Gemini-xx"
    json_summary = llm.invoke({'input': prompt})
    #=================================================================================
    return {'json_summary':json_summary}

def Sending_Email(state):
    try:
        # 환경변수에 앱 비밀번호(2단계 인증) 등록 필요
        password = os.getenv('EMAIL_PASSWORD')
        sender_email = "abc@gmail.com"
        receiver_emails = state.get('receiver_emails')
        
        subscription = state.get('subscription')
        json_summary = state.get('json_summary')
        title = json_summary['title']
        schedule = json_summary['schedule']
        target = json_summary['target']
        application_method = json_summary['application_method']
        important_notes = json_summary['important_notes']

        contents = f"""\
[SSU-pport 알리미] {subscription}의 신규 업데이트 내용입니다.

    제목: {title}
    대상: {target}
    주요일정: {schedule}
    신청방법: {application_method}
    세부내용:
    {important_notes}
        
캘린더연동을 희망하시는 경우 ~
SSU-pport 알리미 서비스 관련 기타 문의사항은 ~
"""

        msg = MIMEMultipart()
        msg['Subject'] = title
        msg['From'] = sender_email
        msg['To'] = receiver_emails
        msg.attach(MIMEText(contents, "plain"))

        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
            server.login(sender_email, password)
            server.sendmail(sender_email, receiver_emails, msg.as_string())
    except Exception as e:
        print(f"메일 전송 실패:\n{e}")

In [ ]:
def OCR_branching_condition(state):
    # image 있으면 OCR_node로
    # image 없으면 Summary_node로
    #==============================================
    # TO DO: OCR 작업 수행 여부를 결정하는 분기 조건
    if True:
        return True
    else:
        return False
    #==============================================

graph = StateGraph(NotifyState)
graph.add_node("OCR_Node", RunnableLambda(OCR))
graph.add_node("Summary_Node", RunnableLambda(Summary))
graph.add_node("Sending_Email_Node", RunnableLambda(Sending_Email))

graph.add_conditional_edges(
    "START",
    OCR_branching_condition,
    {True: "OCR_Node", False: "Summary_Node"}
)
graph.add_edge("OCR_Node", "Summary_Node")
graph.add_edge("Summary_Node", "Sending_Email_Node")
graph.set_finish_point("Sending_Email_Node")

llm_agent = graph.compile()

# from Crawl-Detect Engine
output = llm_agent.invoke(
    {'subscription':None},
    {'receiver_emials':None},
    {'html_processed':None}
)